In [1]:
!pip install --upgrade pip

In [2]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
# !pip install torchcodec
!pip install -U typing_extensions

import torch; torch._dynamo.config.recompile_limit = 64;

In [3]:
%%capture
!pip install --no-deps --upgrade timm # Only for Gemma 3N

In [4]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch

model, processor = FastVisionModel.from_pretrained(
    "blind-assist/gemma-3n-finetune-500train-run-2",
    load_in_4bit = False, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.9: Fast Gemma3N patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A40. Num GPUs = 1. Max memory: 44.448 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

In [5]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 167.9 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 196.9 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.3
    Uninstalling numpy-1.26.3:
      Successfully uninstalled numpy-1.26.3━━━━━ 0/2 [numpy]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [opencv-python]0m [opencv-python]


In [ ]:
from huggingface_hub import login
import os

# Paste your token here (get it from: https://huggingface.co/settings/tokens)
HF_TOKEN = "" 

login(token=HF_TOKEN)

In [7]:
!pip uninstall torchcodec -y

In [8]:
import cv2
import numpy as np
from PIL import Image

def downsample_video(video_path, num_frames=8):
    """Extracts evenly spaced frames for VLM context."""
    
    vidcap = cv2.VideoCapture(video_path)
    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames <= 0: return []
    
    indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    frames = []
    for i in indices:
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, i)
        success, image = vidcap.read()
        if success:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            frames.append(Image.fromarray(image))
    vidcap.release()
    return frames

In [9]:
from unsloth import get_chat_template

processor = get_chat_template(
    processor,
    "gemma-3n"
)

In [12]:
import os
import csv
from pathlib import Path
from transformers import TextStreamer

# Enable Unsloth optimized inference
FastVisionModel.for_inference(model)

# Setup paths
video_folder = "videos"
output_csv = "video_guidance_results.csv"

# Get all video files
video_files = sorted([f for f in os.listdir(video_folder) 
                     if f.endswith(('.mp4', '.avi', '.mov', '.mkv'))])

print(f"Found {len(video_files)} videos to process")

# Instruction for the model
instruction = ("Given the visual input from the user's forward perspective, "
               "generate exactly one short sentence to guide a visually impaired user "
               "by identifying critical obstacles or landmarks, describing their locations "
               "using clock directions relative to the user (12 o'clock is straight ahead), "
               "including relevant details such as size, material, or distance, and giving "
               "one clear action, while prioritizing immediate safety and avoiding any extra explanation.")

# Prepare CSV file
results = []

# Process each video
for idx, video_file in enumerate(video_files, 1):
    video_path = os.path.join(video_folder, video_file)
    print(f"\n[{idx}/{len(video_files)}] Processing: {video_file}")
    
    try:
        # Extract frames from video
        frames = downsample_video(video_path)
        
        # Construct the multimodal message
        messages = [{"role": "user", "content": [{"type": "text", "text": instruction}]}]
        
        # Interleave frames into message content
        for img in frames:
            messages[0]["content"].append({"type": "image", "image": img})
        
        # Apply chat template and process inputs
        input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
        inputs = processor(
            images=frames,
            text=input_text,
            add_special_tokens=False,
            return_tensors="pt",
        ).to("cuda")
        
        # Generate output
        print(f"Generating guidance for {video_file}...")
        output = model.generate(
            **inputs,
            max_new_tokens=128,
            use_cache=True,
            temperature=1.0,
            top_p=0.95,
            top_k=64
        )
        
        # Decode the output
        generated_text = processor.decode(output[0], skip_special_tokens=True)
        
        # Extract only the generated response (remove the prompt)
        # Split by "model" marker and take the last part
        if "model" in generated_text:
            guidance = generated_text.split("model")[-1].strip()
        elif "assistant" in generated_text.lower():
            guidance = generated_text.split("assistant")[-1].strip()
        else:
            guidance = generated_text.strip()
        
        # Remove any remaining newlines and extra whitespace
        guidance = " ".join(guidance.split())
        
        print(f"Output: {guidance}")
        
        # Store result
        results.append({
            "video_id": idx,
            "video_filename": video_file,
            "guidance": guidance
        })
        
    except Exception as e:
        print(f"Error processing {video_file}: {str(e)}")
        results.append({
            "video_id": idx,
            "video_filename": video_file,
            "guidance": f"ERROR: {str(e)}"
        })

# Save results to CSV
print(f"\nSaving results to {output_csv}...")
with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['video_id', 'video_filename', 'guidance']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    writer.writerows(results)

print(f"\nProcessing complete! Results saved to {output_csv}")
print(f"Successfully processed {len([r for r in results if not r['guidance'].startswith('ERROR')])} out of {len(video_files)} videos")

Found 15 videos to process

[1/15] Processing: 20240914_1abe4d6f616ccb2d8e15049136e1656d_4m28s.mp4
Generating guidance for 20240914_1abe4d6f616ccb2d8e15049136e1656d_4m28s.mp4...
Output: a child is passing in front, please pay attention to safety.

[2/15] Processing: 20240918-youtube_short_081e0a96bac802b988a1db9df310ddd1_1min03s.mp4
Generating guidance for 20240918-youtube_short_081e0a96bac802b988a1db9df310ddd1_1min03s.mp4...
Output: a vehicle is passing in the opposite direction at one o'clock direction, please pay attention to safety.

[3/15] Processing: 20240918-youtube_short_11bc0f3b2d7f68e62f5d61a10d2f8897_4min03s.mp4
Generating guidance for 20240918-youtube_short_11bc0f3b2d7f68e62f5d61a10d2f8897_4min03s.mp4...
Output: the wall is clear, walk at ease.

[4/15] Processing: 20240918-youtube_short_1aec7c3e80c13cbb75f75f72333148bf_2min24s.mp4
Generating guidance for 20240918-youtube_short_1aec7c3e80c13cbb75f75f72333148bf_2min24s.mp4...
Output: the current direction of the road is strai

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")